In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset

/home/studio-lab-user/.conda/envs/pyenv-llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# Load dataset
dataset = load_dataset('imdb')

small_train_dataset = dataset["train"].shuffle(seed=42).select([i for i in list(range(5000))])
small_test_dataset = dataset["test"].shuffle(seed=42).select([i for i in list(range(300))])

In [18]:
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize the data
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 300/300 [00:00<00:00, 2537.32 examples/s]


In [19]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [20]:
# Load pre-trained BERT model
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
model.to('cuda')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [21]:
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [22]:
# Define training arguments
training_args = TrainingArguments(
   output_dir='./results-bert',
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=False,
)
 
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [23]:
# Train the model
trainer.train()

Step,Training Loss
500,0.294300


TrainOutput(global_step=626, training_loss=0.2699890685157654, metrics={'train_runtime': 441.8372, 'train_samples_per_second': 22.633, 'train_steps_per_second': 1.417, 'total_flos': 1311874324164864.0, 'train_loss': 0.2699890685157654, 'epoch': 2.0})

In [27]:
trainer.evaluate()

{'eval_loss': 0.3226155638694763,
 'eval_accuracy': 0.8866666666666667,
 'eval_f1': 0.89171974522293,
 'eval_runtime': 4.4428,
 'eval_samples_per_second': 67.525,
 'eval_steps_per_second': 4.277,
 'epoch': 2.0}

In [30]:
trainer.save_model("./bert_sentiment_classifier")

# Load Custom Model/Classifier

In [36]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [39]:
from transformers import pipeline
 
pipe = pipeline(task="sentiment-analysis", model="./bert_sentiment_classifier", tokenizer=tokenizer, device=device)

In [40]:
pipe(["I love this move", "This movie sucks!"])

[{'label': 'LABEL_1', 'score': 0.9678625464439392},
 {'label': 'LABEL_0', 'score': 0.9655263423919678}]